In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import pandas as pd
import gc
import numpy as np
import os

checkpoint_path = 'mlp_imple/cp-{epoch:04d}.ckpt'
model_path = 'mlp_imple.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [2]:
starttime = time.time()
# read in data
mainFrame=pd.read_csv('20062007Small.csv',parse_dates=['date'])
#model = joblib.load(r'C:\Hedge Fund Project\training\modelv1.plk')
mainFrame.set_index(['entityID','date'],inplace=True)
#mainFrame=mainFrame20052018
mainFrame.sort_index(inplace=True)
targets=mainFrame.iloc[:,-7:]
features = mainFrame.iloc[:,:-7]
gc.collect()
endtime = time.time()
print("It takes {}s to load data".format(endtime-starttime))

It takes 52.24381494522095s to load data


In [3]:
maskTrain=(mainFrame.index.get_level_values(1)>='2006') & (mainFrame.index.get_level_values(1)<'2006-10-01')

maskTest=(mainFrame.index.get_level_values(1)>='2006-10-01') & (mainFrame.index.get_level_values(1)<'2007')

x_train=np.array(features[maskTrain])
y_train=np.array(targets['ztargetMedian5'][maskTrain])
x_train[np.isinf(x_train)]=100000000
y_train=y_train*1
y_train=y_train.astype(int)

x_test=np.array(features[maskTest])
y_test = np.array(targets['ztargetMedian5'][maskTest])
x_test[np.isinf(x_test)]=100000000
y_test=y_test*1
y_test=y_test.astype(int)

input_dims=x_train.shape[1]

In [4]:
# Parameters
def create_model():
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_dims,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(32, activation='sigmoid'),
        layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.binary_crossentropy,
                 metrics=['accuracy'])
    return model

# def loadlatestmodel():
#     model = create_model()
#     latest = tf.train.latest_checkpoint(checkpoint_dir)
#     model.load_weights(latest)
    
model = create_model()
# model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               96768     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 133,825
Trainable params: 133,825
Non-trainable params: 0
_________________________________________________________________


In [5]:
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  period=5,
#                                                  verbose=1)
# model.fit(x_train, y_train, batch_size=50, epochs=50, 
#           validation_split=0.1, verbose=1, callbacks=[cp_callback])
model.fit(x_train, y_train, batch_size=50, epochs=50, 
          validation_split=0.1, verbose=1)

Train on 442966 samples, validate on 49219 samples
Epoch 1/50
442966/442966 [==============================] - 48s 109us/sample - loss: 0.7737 - accuracy: 0.5040 - val_loss: 0.6952 - val_accuracy: 0.5001
Epoch 2/50
442966/442966 [==============================] - 37s 84us/sample - loss: 0.6941 - accuracy: 0.5064 - val_loss: 0.6970 - val_accuracy: 0.4754
Epoch 3/50
442966/442966 [==============================] - 37s 83us/sample - loss: 0.6948 - accuracy: 0.5080 - val_loss: 0.6967 - val_accuracy: 0.4903
Epoch 4/50
442966/442966 [==============================] - 37s 84us/sample - loss: 0.6939 - accuracy: 0.5091 - val_loss: 0.6956 - val_accuracy: 0.5087
Epoch 5/50
442350/442966 [============================>.] - ETA: 0s - loss: 0.6937 - accuracy: 0.5054
Epoch 00005: saving model to mlp_imple/cp-0005.ckpt
442966/442966 [==============================] - 37s 84us/sample - loss: 0.6937 - accuracy: 0.5054 - val_loss: 0.6921 - val_accuracy: 0.5140
Epoch 6/50
442966/442966 [===================

Epoch 46/50
442966/442966 [==============================] - 31s 71us/sample - loss: 0.6940 - accuracy: 0.5005 - val_loss: 0.6989 - val_accuracy: 0.4754
Epoch 47/50
442966/442966 [==============================] - 31s 71us/sample - loss: 0.6940 - accuracy: 0.4993 - val_loss: 0.6942 - val_accuracy: 0.4754
Epoch 48/50
442966/442966 [==============================] - 31s 70us/sample - loss: 0.6940 - accuracy: 0.4999 - val_loss: 0.6920 - val_accuracy: 0.5246
Epoch 49/50
442966/442966 [==============================] - 32s 71us/sample - loss: 0.6939 - accuracy: 0.4998 - val_loss: 0.6928 - val_accuracy: 0.5246
Epoch 50/50
442550/442966 [============================>.] - ETA: 0s - loss: 0.6940 - accuracy: 0.5009
Epoch 00050: saving model to mlp_imple/cp-0050.ckpt
442966/442966 [==============================] - 32s 72us/sample - loss: 0.6940 - accuracy: 0.5009 - val_loss: 0.6924 - val_accuracy: 0.5246


In [6]:
result = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(result)
model.save(model_path) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['loss', 'accuracy']
[0.6931644879746548, 0.50387675]
